<a href="https://colab.research.google.com/github/Face-Expression-Recognation-Team-2/Face-Emotion-Recognation/blob/main/Face_Emotion_Recognition_(_Team_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Memasukkan Library

In [ ]:
import numpy as np
import pandas as pd
import os
import zipfile
import shutil
import cv2

#Memasukkan data

In [ ]:
!pip install -q kaggle
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d 'jonathanoheix/face-expression-recognition-dataset'

mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 120M/121M [00:03<00:00, 46.0MB/s]
100% 121M/121M [00:03<00:00, 35.7MB/s]


In [ ]:
dataset_zip = zipfile.ZipFile('face-expression-recognition-dataset.zip', 'r')

dataset_zip.extractall()

dataset_zip.close()

In [ ]:
source_folders = ['/content/images/images/train', '/content/images/images/validation']
destination_folder = '/content/dataset'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for folder in source_folders:
    shutil.move(folder, destination_folder)
    print(f"Folder {folder} berhasil dipindahkan ke {destination_folder}")

print("Success")

Folder /content/images/images/train berhasil dipindahkan ke /content/dataset
Folder /content/images/images/validation berhasil dipindahkan ke /content/dataset
Success


In [ ]:
old_folder = '/content/images'

shutil.rmtree(old_folder)

print("Success")

Success


#Tahap Preprocessing

In [ ]:
# mengambil hanya 50 per emosi
dataset_folder = "dataset"
train_folder = os.path.join(dataset_folder, "train")
validation_folder = os.path.join(dataset_folder, "validation")

images_folder = "images"

emosi = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

def copy_images(source_folder, destination_folder, emosi, num_images):
    for emosi in emosi:

        source_emotion_folder = os.path.join(source_folder, emosi)
        destination_emotion_folder = os.path.join(destination_folder, emosi)

        if not os.path.exists(destination_emotion_folder):
            os.makedirs(destination_emotion_folder)

        images = os.listdir(source_emotion_folder)[:num_images]

        for image in images:
            source_image_path = os.path.join(source_emotion_folder, image)
            destination_image_path = os.path.join(destination_emotion_folder, image)
            shutil.copyfile(source_image_path, destination_image_path)

if not os.path.exists(images_folder):
    os.makedirs(images_folder)

train_images_folder = os.path.join(images_folder, "train")
copy_images(train_folder, train_images_folder, emosi, 50)

validation_images_folder = os.path.join(images_folder, "validation")
copy_images(validation_folder, validation_images_folder, emosi, 50)

print("sukses")

Data telah berhasil disalin ke dalam folder 'images'.
